# Imports

In [18]:
%load_ext autoreload
%autoreload 2
import sys
from nlppen.analisis import Analisis
from nlppen.seleccion import Seleccion
from nlppen.spark_udfs import solo_portanto, spark_get_spacy
from nlppen.sentencias_estructurales import SentenciasEstructurales
from pyspark.sql import SparkSession
from pyspark.sql.functions import length

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Initialize spark context

In [2]:
spark = (SparkSession
         .builder
         .appName("Transforming sentences")
         .config("spark.num.executors", "2")
         .config("spark.executor.memory", "5g")
         .config("spark.executor.cores", "4")
         .config("spark.driver.memory", "5g")
         .config("spark.memory.offHeap.enabled", True)
         .config("spark.memory.offHeap.size", "10g")
         .config("spark.sql.execution.arrow.pyspark.enabled", "true")
         .getOrCreate())

sc = spark.sparkContext
sc.uiWebUrl

21/09/17 17:28:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


'http://7416269fd645:4040'

# Buscar terminos en la sección de por lo tanto de la sentencia

In [3]:
terminos = {
    'seguimiento': [r'\bseguimiento\b'],
    'se ordena': [r'\bse ordena\b'],
    'plan': [r'\bplan\b']
}
seleccion = Seleccion(terminos, spark, parquet_path='./dataset', datasets_path='./datasets/estructurales')
seleccion.filtrar_sentencias(preprocess=solo_portanto)

estructurales = SentenciasEstructurales(seleccion)


# Formar dataset de sentencias estructurales

In [30]:
from pyspark.sql.types import *

columnas = {
    'se ordena PER' : ArrayType(StringType()),
    'se ordena LOC' : ArrayType(StringType()),
    'se ordena ORG' : ArrayType(StringType()),
    'se ordena MISC' : ArrayType(StringType()),
    'se ordena GPE' : ArrayType(StringType())
}
resultado = estructurales.separarSeOrdena(columnas, True, False)
estructurales.sdf = resultado
columnas = {
    'extension sentencia' : IntegerType(),
    'extension por lo tanto' : IntegerType()
}
resultado = estructurales.extraerExtension(columnas)
resultado.toPandas()

,archivo,txt,tema,tema_prob,expediente,fechahora_ext,redactor_ext,tipoResolucion_ext,termino_ext,voto_salvado,...,seguimiento,se_ordena,plan,se_ordena_PER,se_ordena_LOC,se_ordena_ORG,se_ordena_MISC,se_ordena_GPE,extension_sentencia,extension_por_lo_tanto
0,72E91.html,*050139380007CO* Exp: 05-013938-0007-CO Res. N...,MUNICIPALIDAD,0.993193,050139380007CO,2006-03-28 18:21:00,Rodríguez Arroyo,Recurso de Amparo,Con lugar,False,...,0,1,0,"[Ronald Traña Calvo, Patricia Carmona Zeledón]",[Alcalde de la Municipalidad de Santa Ana],[Jefe del Departamento de Cobros de la Municip...,None,None,15786,1503
1,7A4A9.html,*060098620007CO* Exp: 06-009862-0007-CO Res. N...,PETICION,0.753669,060098620007CO,2006-09-13 17:17:00,Cruz Castro,Recurso de Amparo,Con lugar,False,...,0,1,0,"[Marco Vinicio Cordero Quesada, José Daniel Ar...",[Instituto de Desarrollo Agrario],"[Presidente Ejecutivo, Ley de la Jurisdicción ...",None,None,12572,1380
2,7591A.html,*040023800007CO* Exp: 04-002380-0007-CO Res: 2...,AMBIENTE,0.999920,040023800007CO,2006-05-10 16:46:00,Jinesta Lobo,Recurso de Amparo,Con lugar parcial,False,...,0,3,0,"[Oscar Alvarado Bogantes, Fabio Molina Rojas, ...","[Urbanización El Portillo, Turrúcares de Alaju...","[Se ordena 1, Gerente General de la, Mutual Al...",None,None,58750,3136
3,75422.html,*060018790007CO* Exp: 06-001879-0007-CO Res: 0...,TRABAJO,1.000000,060018790007CO,2006-05-12 12:00:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,0,1,0,"[Soledad Chinchilla Arguedas, Subdirectora Gen...","[Ministerio de Educación Pública, Escuela Las ...","[Departamento de Nombramientos, No.2789780]",None,None,14326,1391
4,7BC63.html,*060127740007CO* Exp: 06-012774-0007-CO Res. N...,PRONTA RESOLUCION,0.999759,060127740007CO,2006-11-07 17:32:00,Vargas Benavides,Recurso de Amparo,Con lugar,False,...,0,2,0,"[Jeremías Vargas Chavarría, Franciso Morales H...",None,"[Director Nacional de Pensiones, Ministro de T...",None,None,10363,1912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
840,16a7c.html,Exp.No.3069-M-93 No. 3823-93 SALA CO...,PETICION,0.999991,3069M93,1993-08-06 17:21:00,Mora Mora,Recurso de Amparo,Rechazo de plano,False,...,0,1,0,None,[Corte Suprema de Justicia],"[Juzgado Primero de Trabajo, Presidente]",None,None,4857,379
841,1691a.html,aExp.No.2463-93 No. 3273-93 SALA CONSTITU...,TRABAJO,0.892476,246393,1993-07-09 09:48:00,Mora Mora,Recurso de Amparo,Rechazo de plano,False,...,0,1,0,[San José],None,"[Capitana, Policía Femenina de, Delegado de la...",None,None,3635,455
842,153bc.html,VOTO N 1597-91.- SALA CONSTITUCIONAL DE LA COR...,PETICION,0.999375,1003B91,1991-08-16 09:48:00,Baudrit- y,Recurso de Amparo,Con lugar,False,...,0,1,0,None,[Ministerio],None,None,None,4579,1288
843,14bb1.html,Voto 171-90 Fecha: 13-2-90 Hora: 14:45 Expedie...,SUJETO DE DERECHO PRIVADO,0.424996,469A89,NaT,Rodríguez Vega,Recurso de Amparo,Con lugar,False,...,0,1,0,None,[Sala],None,None,None,6002,808
